In [20]:
# Load and view data

fname = "data.csv"
with open(fname, 'r') as fh:
    lines = fh.readlines()
    
print("Labels: ", lines[0])
print("Sample: ", lines[1])
print("Columns: ", len(lines))

import pandas as pd

df = pd.read_csv(fname)

Labels:  Order,Ticker,Type,Discreet?,YearMonthDayUTC,Address,State,Zip,Zip4,Cars,Notes,UTC,Time_Date,Time,Month,Day,Year,Spaces,Population,Male_Population,Female_Population,Population_Density,5_Year_Projected_Population_Growth,Average_ Family_Size,Family_Households,Households,Age_Below_18,Age_18_Plus,Age_65_Plus,Associate_Degree,Bachelors_Degree,High_Schl_ Grad,PPHouseholds_1,PPHouseholds_2,MT2PPHousehold_2Plus,Per_Capita_Income,Median_Disposable_Household_Income,PCTWhitePop,PCTAIANPop,PCTAPop,PCTBPop,PCTHPIPop,PCTHisPop,PCTMultiracePop,PCTOtherRacePop,PCTMinority,PCTNo_Vehicle,PCT_1_Vehicle,PCT_2_Vehicles,PCTMore_2_Vehicles,PCTCivilian_Unemp,Slot 1,Slot 2,Slot 3,Slot 4,Slot 5,Slot 6,Category,PrePost,DOW,Shifted Month,WMT-Q,Week End,Unique Location,Unique Location & Year,Duplicate?,Year Repeat?,Same Store in Two Years?,Region,Sub Region,Create Year

Sample:  15.0,CMG,CMG_1467,,11APR01185905,7175 W. Lake Mead  Las Vegas,NV,89128,,53.0,04/01/2011,,2011-04-01 11:59:00,11:59,4.0,1.0,2011.0

In [21]:
# Select relevant columns

iwant = ["Region", "Sub Region", "Address", "State", "Zip", "Cars", "YearMonthDayUTC", "PCTNo_Vehicle", "PCT_1_Vehicle","PCT_2_Vehicles","PCTMore_2_Vehicles", 'Median_Disposable_Household_Income', 'Per_Capita_Income', 'Average_ Family_Size', 'Population_Density', 'Male_Population']

for item in df.keys(): 
    if not (item in iwant):
        del df[item]

print(df)

       YearMonthDayUTC                                   Address State  \
0        11APR01185905              7175 W. Lake Mead  Las Vegas    NV   
1        11APR01172545      1715 North Town East Blvd.  Mesquite    TX   
2        11APR01190010         18951 Brookhurst  Fountain Valley    CA   
3        11APR01172543                       4901 W. Park  Plano    TX   
4        11APR01172545        7700 N. Central Expressway  Dallas    TX   
5        11APR01190024        12359 Seal Beach Blvd.  Seal Beach    CA   
6        11APR01174959                16680 N. 83rd Ave.  Peoria    AZ   
7        11APR01172543         1009 N. Central Expressway  Plano    TX   
8        11APR01190024       16241 Beach Blvd.  Huntington Beach    CA   
9        11APR01190023             5310 Lakewood Blvd.  Lakewood    CA   
10       11APR01175000         5880 W. Thunderbird Rd.  Glendale    AZ   
11       11APR01193126  990 Serramonte Boulevard  Suite D  Colma    CA   
12       11APR01190010             230

In [22]:
# Split data into 120 chunks

import numpy as np

s = 900
t = int(296703 / s) +1

chunks = df.groupby(np.arange(len(df)) // s)
print(df.shape, "=>", len(chunks), "x", "({}, {})".format(s, 16))

indices = list(map(lambda x: x*s, range(0,t)))
parcels = list(zip(indices, chunks)) # list of (start_index, df_chunk) tuples

(296702, 16) => 330 x (900, 16)


In [30]:
import boto3
import os
import requests
import paramiko
from time import sleep

### Config ###
total_servers = 1#119
servers_per_batch = 1#12


# Create tarball for server from tuple
#
# Result:
#     <#>/
#       - src.tar.gz
#       - src/
#           - runscript.sh
#           - scraper.py
#           - data.csv
#

def create_tarball(tup):
    # 1. Create runscript
    ind, dfc = tup
    tmpdir = str(ind)+"/src/"
    cmd = "mkdir -p "+tmpdir
    os.system(cmd)
    
    # 1. Copy runscript
    cmd = "cp runscript.sh "+tmpdir
    os.system(cmd)
    
    # 2. Create datafile 
    datafile = tmpdir+"data.csv"
    dfc.to_csv(path_or_buf=datafile)
    
    # 3. Copy scraper program
    cmd = "cp scraper.py "+tmpdir
    os.system(cmd)
    
    # 4. Compress runscript.sh, data.csv, scraper.py
    cmd = "cd "+str(ind)+" && tar czf src.tar.gz src/"
    os.system(cmd)


### Batches ###

# initialize aws client
ec2 = boto3.resource('ec2')

#servers = range(0,total_servers)
start_inds = range(0, total_servers, servers_per_batch)

# create servers in batches
#if False:
for ind in [0]:#start_inds:
    # compress tarball
    for i in range(ind, ind+servers_per_batch):
        if i <= total_servers:
            create_tarball(parcels[i][1])


    # start 12 servers (1-12, 13-24, etc.)
    print("starting servers: {}-{}".format(ind, ind+servers_per_batch))
    batch = ec2.create_instances(
        ImageId="ami-9be6f38c",
        InstanceInitiatedShutdownBehavior="terminate",
        InstanceType="t2.nano",
        KeyName="cluster", 
        MinCount=servers_per_batch,
        MaxCount=servers_per_batch)

    # wait for all servers to start and collect ip addresses
    names = []
    for instance in batch:
        print("server #{} not up yet...".format(instance.id))
        instance.wait_until_running()
        instance.reload()
        print((instance.id, instance.state, instance.public_dns_name, instance.public_ip_address))
        names.append(instance.public_ip_address)
    print("all batch servers up")
 
    
    print("servers: {}".format(names))
    
    print("waiting for initialization...")
    # send processing tasks via scp   
    for instance in batch:
        try:
            res = os.system("scp -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i cluster.pem {}/src.tar.gz ec2-user@{}:src.tar.gz".format(ind, instance.public_ip_address))
            while res: # not equal to success case 0
                print("waiting...")
                sleep(60) # have to wait here for first in batch to initialize
                res = os.system("scp -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i cluster.pem {}/src.tar.gz ec2-user@{}:src.tar.gz".format(ind, instance.public_ip_address))          
            
        except Exception as e:
            print("could not upload data to {}: {}".format(instance.id,e))
    
    print("uploads complete")
    
#if True:
    for instance in batch: 
        # ssh in and start task
        print("starting task...")
        host = instance.public_ip_address
        cmd = 'echo "{}" | ./remoteCmd.sh'.format(host)
        os.system(cmd)
        print("task started")
        
        
    print("batch initialization complete!")
    
    #addresses = list(map(lambda x: "http://"+x, names))
    #print("collecting from:")
    #print(names)
    
    # collect completed tasks and shut down
    index = 0
    for instance in batch:
        print("collecting from: {}".format(instance.public_ip_address))
        wdir = os.getcwd()
        cmd = "cd {}/{}/ && wget {}".format(wdir, ind+index, "http://"+instance.public_ip_address+":8000/output.tar.gz")
        not_done = os.system(cmd)
        
        counter = 0
        # wait for done
        while not_done:
            print("waiting... {}".format(counter))
            counter = counter + 1
            
            sleep(180)
            not_done = os.system(cmd)
                
        index = index + 1
        print("server #{} complete".format(ind+index))
    
    print("terminating batch in 15...")
    sleep(15)
    for instance in batch:
        instance.terminate()
    print("batch complete!")
    

starting servers: 0-1
server #i-0c949d5dfd5f1fa81 not up yet...
('i-0c949d5dfd5f1fa81', {'Code': 16, 'Name': 'running'}, 'ec2-54-210-248-228.compute-1.amazonaws.com', '54.210.248.228')
all batch servers up
servers: ['54.210.248.228']
waiting for initialization...
waiting...
uploads complete
starting task...
task started
batch initialization complete!
collecting from: 54.210.248.228
waiting... 0
waiting... 1
server #1 complete
terminating batch in 15...
batch complete!


In [31]:
# merge 119 output files

os.system("mkdir output")

def collect(num):
    cmd = "cd {}/ && tar xzf output.tar.gz".format(num)
    os.system(cmd)
    
    cmd = "mv {}/output.csv output/{}.csv".format(num, num)
    os.system(cmd)
    
    print("{} ok".format(num))

def merge(num):
    fname = "output/{}.csv".format(num)
    return pd.read_csv(fname)
    
frames = []    
for i in range(0, 1):
    collect(i)
    frames.append(merge(i))

result = pd.concat(frames)
print(result)

0 ok
     Unnamed: 0 YearMonthDayUTC                                       Address  \
0           812   12APR11175013                       2082 Ford Pkwy St. Paul   
1            97   11APR12192925            12359 Seal Beach Blvd.  Seal Beach   
2           676   12APR04160802                  3496 Erie Blvd East Syracuse   
3           517   12APR19165325                 5410 Dixie Highway  Fairfield   
4           269   11APR30191553                  7440 Laguna Blvd.  Elk Grove   
5           452   12APR13181850       3026 New Center Point  Colorado Springs   
6           711   12APR05172307                    3771 Belt Line Rd. Addison   
7           309   12APR03174451                      2082 Ford Pkwy  St. Paul   
8           147   11APR16184218                  7175 W. Lake Mead  Las Vegas   
9           734   12APR06173624                 8700 N. Flintlock Rd. Liberty   
10          857   12APR14160156                 6317 Muliplex Dr. Centreville   
11          821   12APR

In [ ]:
#a = df[['Address', 'State']]
import requests

df['Lat'] = 0.0
df['Lon'] = 0.0
fails = []
numfails = 0
for i in range(0,len(df)):
   
    try:
        s = str(df.ix[i]['State'])
        a = str(df.ix[i]['Address'])
    
        stub = 'http://maps.google.com/maps/api/geocode/json?address="'
        query = stub + "+".join(a.split(' ')) + "+" + s
    
        r = requests.get(query).json()
        if r["status"] == "OK":
            df.set_value(i, 'Lon', r["results"][0]['geometry']['location']['lng'])
            df.set_value(i, 'Lat', r["results"][0]['geometry']['location']['lat'])
            if i % 1000 == 0:
                print("index #{}".format(i))
            
        else:
            numfails = numfails+1
            print(numfails)
            fails.append(query)
    except Exception as e:
        print(e)
        print("failure index: {}".format(i))

print("cannot find coordinates for these addresses:\n{}".format(fails))
df.to_csv(path_or_buf="cars.csv")

print(df['Lat'])